In [168]:
using MultivariatePolynomials
using MultivariateMoments
using SemialgebraicSets
using JuMP
using PolyJuMP
using SumOfSquares
using DynamicPolynomials
using MosekTools

In [169]:
include("code/data/dubins.jl")

Polynomial{true, Float64}[]

In [3]:
function compute_moments(parameter_list)
    # the range of parameters is assumed to be scaled to [-1, 1]^N
    # compute moments
    # moments[i]: the moment of paraList[i]
    moments = map(parameter_list) do var
        expList = exponents(var)
        acc = 1.0
        for i=1:length(expList)
            upperval = (1.0 / (expList[i] + 1.0)) * (1.0)^(expList[i] + 1.0)
            lowerval = (1.0 / (expList[i] + 1.0)) * (-1.0 )^(expList[i] + 1.0)
            acc *= (upperval - lowerval)
        end
        return acc
    end
    return moments
end

compute_moments (generic function with 1 method)

In [174]:
# computeR: return p(a), p(a) <= 0 underapproximates the valid set R
# parameter_list is the monomials used to form p(a)
function computeR_p(H, g, parameter_list, box_constriants)
    # @show H g box_constriants
    # H[i] is a list of polynomials representing the requirements,
    # g is the polynomial representing the result
    # the definition of valid set R: H[i] <= 0 implies g <= 0
    
    # m: the model
    m = SOSModel(optimizer_with_attributes(Mosek.Optimizer, "QUIET" => true))
    # p[i]: the (unknown) coefficients of p(a), p[i] corresponds to the monomial paraList[i]
    @variable(m, p[1:length(parameter_list)])
    poly = sum(p[i] * parameter_list[i] for i=1:length(p))

    # S is the feasible range (i.e. H[i] <= 0, a[i] and x[i] satisfies box_constraints)
    # S_box: only the box constaints (i.e. S without H[i] <= 0)
    
    # adding -H[i] >= 0
    S = basicsemialgebraicset(FullSpace(), -H)
    # adding box_constraints
    S_box = basicsemialgebraicset(FullSpace(), -box_constriants)
    
    S = intersect(S, S_box)
    
    
    # ϵ is the relaxation used in Putinar's Positivstellsatz,
    # it makes the hierarchy converges faster
    ϵ = 0.01; M = 10.0
    
    #the first constraint in m
    @constraint(m, poly-g + ϵ >= 0, domain = S, maxdegree = 6)
    
    #the second constriant in m
    @constraint(m, poly + ϵ >= -M, domain = S_box, maxdegree = 6)
    
    moments = compute_moments(parameter_list)
    #obj: objective function, minimizing the integral of p(a) over S
    obj = sum(moments[i] * p[i] for i=1:length(p))
    @objective(m, Min, obj)
    
    optimize!(m)
    @show termination_status(m)
    pvalue = value.(p)
    return sum(pvalue[i] * parameter_list[i] for i = 1:length(parameter_list))
end

computeR_p (generic function with 1 method)

In [5]:
function scale_range(poly, vars, range_before, range_after)
    old_vars = map(1:length(vars)) do i
        length_before = range_before[i][2] - range_before[i][1]
        length_after = range_after[i][2] - range_after[i][1]
        multiplier = length_before / length_after
        multiplier * (vars[i] - range_after[i][1]) + range_before[i][1]
    end
    scaled_poly = subs(poly, vars => old_vars)
    return scaled_poly
end

scale_range (generic function with 1 method)

In [6]:
function make_box_constraints(a, x, rangeA, rangeX)
    # making box constraints
    ADIM = length(rangeA) 
    XDIM = length(rangeX)
    greater_than_a = [rangeA[i][1] - a[i] for i in 1:ADIM]
    less_than_a = [a[i] - rangeA[i][2] for i in 1:ADIM]
    greater_than_x = [rangeX[i][1] - x[i] for i in 1:XDIM]
    less_than_x = [x[i] - rangeX[i][2] for i in 1:XDIM]

    box_constraints = [greater_than_a; less_than_a; greater_than_x; less_than_x]
    return box_constraints
end

make_box_constraints (generic function with 1 method)

In [7]:
function eval_poly(a, p, var)
    return leadingcoefficient(subs(p, a => var))
end

eval_poly (generic function with 1 method)

In [9]:
notloopcond = Polynomial{true, Float64}[]

Polynomial{true, Float64}[]

In [175]:
# scale the range of parameters to [-1, 1]
rangeA_scaled = fill((-1, 1), length(rangeA))
template_scaled = scale_range(template, a, rangeA, rangeA_scaled)

# approximation polynomial p_i will be chosen from 
# linear combinations of these monomials
parameter_list = monomials(Tuple(a), 0:ADEG)
    
# NextX: the mapped result of the loop body
Next_x = [F_next[i](x) for i in 1:length(F_next)]
# templateNext: what template would be after such a mapping
Template_scaled_next = [subs(template_scaled, x => Next_x[i]) for i in 1:length(F_next)]
    
box_constraints = make_box_constraints(a, x, rangeA_scaled, rangeX);

In [ ]:
computeR(H, g) = computeR_p(H, g, parameter_list, box_constraints)

In [176]:
computeR(H, g) = computeR_p(H, g, parameter_list, box_constraints)
    
p_pre = computeR([precond;additional_invariants], template_scaled)
p_conds = [computeR([[template_scaled, Cond[i]]; loopcond; additional_invariants], Template_scaled_next[i]) for i in 1:length(Cond)]
p_post = computeR([[template_scaled]; notloopcond; additional_invariants], postcond)
#p_post = computeR([[template_scaled]; additional_invariants], postcond)

p_list = [[p_pre];p_conds;[p_post]]

termination_status(m) = MathOptInterface.OPTIMAL
termination_status(m) = MathOptInterface.OPTIMAL
termination_status(m) = MathOptInterface.OPTIMAL


3-element Vector{Polynomial{true, Float64}}:
 -0.013344954559285813a₁⁴ - 2.9013470941040853e-11a₁³a₂ + 4.8163229130652916e-8a₁²a₂² - 2.9635930431746084e-11a₁a₂³ + 8.413872624217831e-8a₂⁴ - 1.0645601259679819e-7a₁³ - 6.302510690024504e-11a₁²a₂ + 1.73641157247083e-8a₁a₂² - 2.0940028516410615e-11a₂³ + 0.03328852618517408a₁² + 1.730194038830356e-11a₁a₂ - 9.390711534727876e-8a₂² - 0.024999831350335643a₁ + 1.999999999916376a₂ - 0.004943647302993286
 4.1527452834623597e-5a₁⁴ - 0.0003362765417953351a₁³a₂ - 0.00948287323067512a₁²a₂² - 0.14545625900787604a₁a₂³ - 2.7791953985110838a₂⁴ + 0.000620337735918739a₁³ + 0.007804208357337266a₁²a₂ - 0.26716353317215036a₁a₂² - 7.254796042436476a₂³ + 0.016831951069121713a₁² + 0.2302712725800459a₁a₂ - 2.6952574937325777a₂² + 0.19209102466819664a₁ + 1.839971628961851a₂ + 0.8808531725361263
 0.0005984319422470565a₁⁴ + 0.0014005266352129203a₁³a₂ + 0.012796155365927021a₁²a₂² - 0.16441660921824125a₁a₂³ - 1.9665496032379177a₂⁴ - 0.0014285363619307837a₁³ - 0.0385109

In [160]:
postcond = -((x[1] - 3)^2 + x[2]^2 - 0.25) 
p_post = computeR([[template_scaled]; notloopcond; additional_invariants], postcond)
p_list = [[p_pre];p_conds;[p_post]]

has_values(m) = true


3-element Vector{Polynomial{true, Float64}}:
 0.021650569342070023a₁² - 1.555927650509029e-12a₁a₂ - 1.1280265735517452e-8a₂² - 0.02499991091477911a₁ + 1.999999999904377a₂ - 0.0027831811861453006
 1.828097647331188e-5a₁² - 1.1597723277878178e-5a₁a₂ + 2.2096030291994417e-5a₂² + 1.087826813728331e-5a₁ - 0.00021794922449959827a₂ - 0.00989642922563894
 0.029556438726938524a₁² - 0.02201594364910915a₁a₂ - 1.4636961505181014a₂² + 0.06336770668746328a₁ - 4.108447542146975a₂ - 4.423394517011363

In [89]:
using BlackBoxOptim

In [90]:
# searching for a_0 such that p_list[i] <= 0 for all i
function search_valid_parameter_bboptim(p_list, a, rangeA)
    tolerance = 0.0
    p_funs = map(p -> (var -> leadingcoefficient(subs(p, a => var))), p_list)    
    function fitness_schaffer(var)
        p_at = map(pf -> pf(var), p_funs)
        return Tuple(p_at)
    end
    
    result = bboptimize(fitness_schaffer; Method=:borg_moea,
            FitnessScheme=ParetoFitnessScheme{length(p_funs)}(is_minimizing=true),
            SearchRange=(-1.0, 1.0), NumDimensions=length(a), ϵ=0.05,
            MaxTime=1.0, TraceInterval=1.0, TraceMode=:silent);
    
    for sol in pareto_frontier(result)
        failed = false
        for p_val in fitness(sol)
            if p_val > tolerance
                failed = true
                break
            end
        end
        if !failed
            return (success = true, parameter = params(sol))
        end
    end
    return (success = false, parameter = nothing)
end       

search_valid_parameter_bboptim (generic function with 1 method)

### searching for a_0 such that p_list[i] <= 0 for all i
function search_valid_parameter_sos(p_list, a, rangeA, )
    MAX_TRY = 10
    c = ones(length(p_list))

    for i in 1:MAX_TRY
        msol = SOSModel(optimizer_with_attributes(Mosek.Optimizer, "QUIET" => true))
        Sa = basicsemialgebraicset(FullSpace(), 
            [[a[i] - rangeA[i][1] for i in 1:ADIM];[rangeA[i][2] - a[i] for i in 1:ADIM]])
        @variable(msol, lb)
        @objective(msol, Max, lb)
        
        opt = sum([c[i] * p_list[i] for i in 1:length(p_list)])
        consol = @constraint(msol, opt >= lb, domain = Sa)
        
        
        optimize!(msol)
        if objective_value(msol) > 1e-3
            continue;
        end
        
        # extract the minimizer
        ν = moment_matrix(consol)
        μ = extractatoms(ν, 1e-3)
        if μ == nothing
            # TODO: increase `maxdegree` in consol
            continue
        end
        atoms = μ.atoms
        minimizer = atoms[1].center
        
        failed = false
        for i in 1:length(p_list)
            #using a combination of `subs` and `leadingcoefficient` to evaluate
            p_at = eval_poly(a, p_list[i], minimizer)
            if p_at > 1e-3
                c[i] *= 2
                failed = true
            end
        end
        
        if !failed
            return (success = true, parameter = minimizer)
        end
    end
    
    #In case minimizer extraction fails too many times
    failed = false
    for i in 1:length(p_list)
        p_at = eval_poly(a, p_list[i], zeros(length(a)))
        if p_at > 1e-3
            @show p_list[i] p_at
            failed = true
            break
        end
    end
    if !failed
        return (success = true, parameter = zeros(length(a)))
    else
        return (success = false, parameter = nothing)
    end
end         

In [177]:
result = search_valid_parameter_bboptim(p_list, a, rangeA_scaled)

(success = false, parameter = nothing)

In [163]:
if result.success
    invariant = subs(template_scaled, a => result.parameter)
else
    invariant = nothing
end

x₁² + 0.965739116444029x₂² - 1.931478232888058x₂ - 1.675300999852464

In [154]:
result_fitness = map(p -> eval_poly(a, p, result.parameter), p_list)

3-element Vector{Float64}:
 -2.001769697884541
 -0.00969137055437036
 -0.2657914505171508

In [199]:
result_fitness = map(p -> eval_poly(a, p, [0, -0.545]), p_list)

3-element Vector{Float64}:
 -1.0949436677237472
  0.006714865985127827
 -1.892431790242329